In [1]:
%load_ext autoreload
%autoreload 2
import torch
import transformers
from transformers import AutoModel
from transformers import AutoModelForCausalLM, AutoTokenizer 
from openai import OpenAI
from openai_key import OPENAI_API_KEY
from conf import *

In [2]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
model = AutoModelForCausalLM.from_pretrained(MODEL_1)
tokenizer = AutoTokenizer.from_pretrained(MODEL_1)

In [4]:
def M_local(*inputs):
  concatenated_input = "\n".join(inputs)
  tokens = tokenizer.encode(concatenated_input, return_tensors="pt")
  generated_ids = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_new_tokens=100)
  output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return output[len(concatenated_input):], output

def M_openai(*inputs):
  concatenated_input = "\n".join(inputs)
  response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": concatenated_input}
  ], max_tokens=1000)
  return response.choices[0].message.content

M = M_local

In [5]:
out = M("This is", "a test")
out

(' of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++',
 'This is\na test of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++ standard library.\nThis is a test of the new\nversion of the\nC++')

In [6]:
p_gen = f"""You are a programmer. You solve programming problems. 

Here is an example:
Input: Write code to print 'Hello World'

Response: print('Hello World')
"""

p_fb = f"""You are a code reviewer. You will be given a problem and a solution and you should give feedback on the solution.
Here is an example:
Input: Write code to print 'Hello World'

import sys
sys.stdout.write('Hi sir!!')

Response: The code does not print 'Hello World' as required. Additionally, you should use print() instead of sys.stdout.write().
"""

p_refine = f"""You improve upon code given a problem, a solution and some feedback on the solution.

Here is an example:
Input: Write code to print 'Hello World'

import sys
sys.stdout.write('Hi sir!!')

The code does not print 'Hello World' as required. Additionally, you should use print() instead of sys.stdout.write().

Response: print('Hello World')
"""

In [7]:
x = f"""Input: Write code to calculate the sum of 1, 2, ..., N
"""

y0 = M(p_gen, x)
#fb = M(p_fb, x, y0)
#y1 = M(p_refine, x, y0, fb)

In [8]:
print(y0)

('Response: N = N + 1\n\nInput: Write code to print the sum of 1, 2,..., N\n\nResponse: N = N + 1\n\nInput: Write code to print the sum of 1, 2,..., N\n\nResponse: N = N + 1\n\nInput: Write code to print the sum of 1, 2,..., N\n\nResponse: N = N + 1\n\nInput: Write code to print the sum of', "You are a programmer. You solve programming problems. \n\nHere is an example:\nInput: Write code to print 'Hello World'\n\nResponse: print('Hello World')\n\nInput: Write code to calculate the sum of 1, 2,..., N\n\nResponse: N = N + 1\n\nInput: Write code to print the sum of 1, 2,..., N\n\nResponse: N = N + 1\n\nInput: Write code to print the sum of 1, 2,..., N\n\nResponse: N = N + 1\n\nInput: Write code to print the sum of 1, 2,..., N\n\nResponse: N = N + 1\n\nInput: Write code to print the sum of")


In [9]:
fb = M(p_fb, x, y0)

TypeError: sequence item 2: expected str instance, tuple found

In [ ]:
print(fb)

The solution you provided looks correct and efficient for calculating the sum of numbers from 1 to N. Good job!

Just a minor improvement, you can update the function name to `calculate_sum_of_numbers` or `calculate_sum` to make it more descriptive.

Overall, great job!


In [ ]:
y1 = M(p_refine, x, y0, fb)

In [ ]:
fb

'The solution you provided looks correct and efficient for calculating the sum of numbers from 1 to N. Good job!\n\nJust a minor improvement, you can update the function name to `calculate_sum_of_numbers` or `calculate_sum` to make it more descriptive.\n\nOverall, great job!'

In [ ]:
print(y1)

Input: Write code to find the square root of a given number using the Newton-Raphson method.

def square_root(number):
    approximation = number / 2
    while True:
        better_approximation = (approximation + number/approximation) / 2
        if abs(approximation - better_approximation) < 0.0001:
            break
        approximation = better_approximation
    return better_approximation

Feedback: The code works correctly and efficiently. However, it would be better to add some comments to explain the logic behind the Newton-Raphson method and the purpose and parameters of the function.

Response:
```
def square_root(number):
    # Initialize approximation with half of the number
    approximation = number / 2
    
    # Iterate until the approximation converges to the square root
    while True:
        # Calculate a better approximation using Newton-Raphson method
        better_approximation = (approximation + number/approximation) / 2
        
        # Check if the approxi